# Příklad ukázkového agenta pro rezervaci hotelů a letů

Toto řešení vám pomůže rezervovat letenky a hotel. Scénář zahrnuje cestu z Londýna Heathrow (LHR) 20. února 2024 do New Yorku (JFK) s návratem 27. února 2025, let v ekonomické třídě pouze s British Airways. Chci pobyt v hotelu Hilton v New Yorku, prosím, uveďte náklady na let a hotel.


# Inicializace služby Azure AI Agent a získání konfiguračních informací z **.env**

### **.env** 

Vytvořte soubor .env 

Soubor **.env** obsahuje připojovací řetězec služby Azure AI Agent, model používaný AOAI a odpovídající API služby Google API Search, ENDPOINT atd.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Název nasazení modelu služby Azure AI Agent"

[**NOTE**] Budete potřebovat model s limitem 100 000 tokenů za minutu (Rate Limit) a limitem 600 požadavků za minutu (Request per minute).

  Model můžete získat v Azure AI Foundry - Model and Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Připojovací řetězec projektu služby Azure AI Agent"

  Připojovací řetězec projektu najdete v přehledu projektu na obrazovce AI ​​Foundry Portal.

- **SERPAPI_SEARCH_API_KEY** = "Vaše API KEY pro SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Váš ENDPOINT pro SERPAPI Search"

Pro získání názvu nasazení modelu a připojovacího řetězce projektu služby Azure AI Agent je nutné vytvořit službu Azure AI Agent. Doporučuje se použít [tuto šablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) pro přímé vytvoření. (***Poznámka:*** Služba Azure AI Agent je aktuálně nastavena v omezeném regionu. Doporučuje se odkazovat na [tento odkaz](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) pro nastavení regionu.)

Agent potřebuje přístup k SERPAPI. Doporučuje se zaregistrovat pomocí [tohoto odkazu](https://serpapi.com/searches). Po registraci můžete získat jedinečný API KEY a ENDPOINT.


# Nastavení

Pro spuštění tohoto notebooku se ujistěte, že jste nainstalovali potřebné knihovny pomocí příkazu `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Vaše verze Semantic Kernel by měla být alespoň 1.27.2.


Načtěte nastavení a zdroje svého souboru .env, ujistěte se, že jste přidali své klíče a nastavení a vytvořili místní soubor .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Přihlášení do Azure

Nyní se musíte přihlásit do Azure. Otevřete terminál a spusťte následující příkaz:

```bash
az login
```

Tento příkaz vás vyzve k zadání vašich přihlašovacích údajů do Azure, což umožní správné fungování služby Azure AI Agent.


# Vysvětlení:
Toto je proměnná, která ukládá API klíč pro přístup k API službě SERP (Search Engine Results Page). API klíč je jedinečný identifikátor používaný k ověření požadavků spojených s vaším účtem.

Účel: Účelem tohoto řádku je uložit API klíč do proměnné, aby mohl být použit k ověření požadavků na službu SERP API. API klíč je nezbytný pro přístup ke službě a provádění vyhledávání.  
Jak získat API klíč pro SERP: Chcete-li získat API klíč pro SERP, postupujte podle těchto obecných kroků na https://serpapi.com (konkrétní kroky se mohou lišit v závislosti na používané službě SERP API):

Vyberte si službu SERP API: Existuje několik dostupných služeb SERP API, například SerpAPI, Google Custom Search JSON API a další. Vyberte si tu, která nejlépe vyhovuje vašim potřebám.

Zaregistrujte si účet: Přejděte na webové stránky vybrané služby SERP API a zaregistrujte si účet. Možná budete muset poskytnout základní informace a ověřit svou e-mailovou adresu.

Vytvořte API klíč: Po registraci se přihlaste ke svému účtu a přejděte do sekce API nebo na ovládací panel. Hledejte možnost vytvoření nebo vygenerování nového API klíče.  
Zkopírujte API klíč do svého souboru .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Vysvětlení:
BASE_URL: Toto je proměnná, která uchovává základní URL pro koncový bod SERP API. Název proměnné BASE_URL je konvence, která označuje, že tato URL je výchozím bodem pro provádění požadavků na API.
'https://serpapi.com/search':

Toto je skutečný řetězec URL přiřazený proměnné BASE_URL. Reprezentuje koncový bod pro provádění vyhledávacích dotazů pomocí SERP API.

# Účel:
Účelem tohoto řádku je definovat konstantu, která uchovává základní URL pro SERP API. Tato URL bude použita jako výchozí bod pro sestavování požadavků na API za účelem provádění vyhledávacích operací.

# Použití:
Definováním základní URL v proměnné ji můžete snadno znovu použít v celém svém kódu, kdykoli potřebujete provést požadavky na SERP API. To činí váš kód lépe udržovatelným a snižuje riziko chyb způsobených tvrdým zakódováním URL na více místech. Aktuální příklad je https://serpapi.com/search?engine=bing, který používá Bing search API. Různé API lze vybrat na https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Vysvětlení:

Zde se nachází váš pluginový kód.

Definice třídy: `class BookingPlugin`: Definuje třídu s názvem BookingPlugin, která obsahuje metody pro rezervaci hotelů a letů.

Metoda pro rezervaci hotelu:

- `@kernel_function(description="booking hotel")`: Dekorátor, který popisuje funkci jako kernelovou funkci pro rezervaci hotelů.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definuje metodu pro rezervaci hotelů s anotovanými parametry a návratovým typem.

Metoda sestavuje slovník parametrů pro požadavek na rezervaci hotelu a odesílá GET požadavek na SERP API. Kontroluje stav odpovědi a vrací vlastnosti hotelu, pokud je požadavek úspěšný, nebo None, pokud požadavek selhal.

Metoda pro rezervaci letu:

- `@kernel_function(description="booking flight")`: Dekorátor, který popisuje funkci jako kernelovou funkci pro rezervaci letů.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definuje metodu pro rezervaci letů s anotovanými parametry a návratovým typem.

Metoda sestavuje slovníky parametrů pro požadavky na odletový a návratový let a odesílá GET požadavky na SERP API. Kontroluje stav odpovědi a přidává informace o letu do výsledného řetězce, pokud je požadavek úspěšný, nebo vypisuje chybovou zprávu, pokud požadavek selhal. Metoda vrací výsledný řetězec obsahující informace o letu.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Vysvětlení:
Importování modulů: Importujte potřebné moduly pro Azure přihlašovací údaje, AI agenta, obsah zpráv v chatu, roli autora a dekorátor funkcí jádra.

Asynchronní správce kontextu: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Nastaví asynchronní správce kontextu pro práci s Azure přihlašovacími údaji a vytvoření klienta AI agenta.

Název agenta a instrukce:
- `AGENT_NAME = "BookingAgent"`: Definuje název agenta.
- `AGENT_INSTRUCTIONS = """..."""`: Poskytuje podrobné instrukce pro agenta, jak zpracovávat požadavky na rezervace.

Vytvoření definice agenta: `agent_definition = await client.agents.create_agent(...)`: Vytvoří definici agenta se specifikovaným modelem, názvem a instrukcemi.

Vytvoření AzureAI agenta: `agent = AzureAIAgent(...)`: Vytvoří AzureAI agenta pomocí klienta, definice agenta a definovaného pluginu.

Vytvoření vlákna: `thread: AzureAIAgentThread | None = None`: Vytvoří vlákno pro agenta. Není nutné nejprve vytvořit vlákno - pokud je poskytnuta hodnota `None`, nové vlákno bude vytvořeno při prvním volání a vráceno jako součást odpovědi.

Vstupy uživatele: `user_inputs = ["..."]`: Definuje seznam vstupů uživatele, které má agent zpracovat.

Ve finálním bloku smažte vlákno a agenta, abyste uvolnili zdroje.


# Autentizace

Třída `DefaultAzureCredential` je součástí Azure SDK pro Python. Poskytuje výchozí způsob autentizace s Azure službami. Pokouší se autentizovat pomocí několika metod v konkrétním pořadí, jako jsou proměnné prostředí, spravovaná identita a přihlašovací údaje Azure CLI.

Asynchronní operace: Modul aio naznačuje, že třída DefaultAzureCredential podporuje asynchronní operace. To znamená, že ji můžete použít s asyncio pro provádění neblokujících autentizačních požadavků.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby AI pro překlady [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
